**bnt-examples-included-in-distro**

The BNT (Bayes Net Toolbox by Kevin Murphy et al) distribution at GitHub contains a folder ```bnt/BNT/examples``` with many examples presented as m-files. This notebooks runs a few of those examples.

BNT is written in Matlab. Running this notebook requires that BNT, Python, Octave and octavemagic be installed on your computer.

In [1]:
import os
import sys
nb_dir = os.getcwd()
print(nb_dir)
os.chdir('../../')
qfog_path = os.getcwd()
print(qfog_path)
sys.path.insert(0,qfog_path)

/home/jupyter/Notebooks/Quantum/quantum-fog/jupyter-notebooks/murphys-bnt
/home/jupyter/Notebooks/Quantum/quantum-fog


In [2]:
bnt_path = '/home/jupyter/Notebooks/Classical/bnt'

In [3]:
import oct2py
%load_ext oct2py.ipython

In [4]:
%%octave
do_braindead_shortcircuit_evaluation (1)
warning('off', 'Octave:possible-matlab-short-circuit-operator')

In [5]:
%octave_push bnt_path
# genpath(dir) grows dir to list of all files inside it (recursive)
%octave addpath(genpath(bnt_path))  

In [6]:
fdir = bnt_path + '/BNT/examples/static/' 
%ls $fdir

Belprop/                    discrete2.m    lw1.m      nodeorderExample.m
brainy.m                    discrete3.m    mfa1.m     qmr1.m
Brutti/                     dtree/         Misc/      qmr2.m
burglar-alarm-net.lisp.txt  fa1.m          mixexp1.m  sample1.m
burglary.m                  fgraph/        mixexp2.m  SCG/
cg1.m                       gaussian1.m    mixexp3.m  softev1.m
cg2.m                       gaussian2.m    Models/    softmax1.m
cmp_inference_static.m      gibbs_test1.m  mog1.m     sprinkler1.m
CVS/                        HME/           mpe1.m     StructLearn/
discrete1.m                 learn1.m       mpe2.m     Zoubin/


In [7]:
def do_file(fname):
    fpath =  fdir + fname
    print(fpath)
    with open(fpath) as f:
        print(f.read()) # prints whole file
    print('_________________________output______________________________')
    fname1 = fname[:-2] # removes '.m'  
    %octave $fname1

In [8]:
do_file('burglary.m')

/home/jupyter/Notebooks/Classical/bnt/BNT/examples/static/burglary.m
% Burglar alarm example

N = 5;
dag = zeros(N,N);
E = 1; B = 2; R = 3; A = 4; C = 5;
dag(E,[R A]) = 1;
dag(B,A) = 1;
dag(A,C)=1;

% true = state 1, false = state 2
ns = 2*ones(1,N); % binary nodes
bnet = mk_bnet(dag, ns);

bnet.CPD{E} = tabular_CPD(bnet, E, [0.1 0.9]);
bnet.CPD{B} = tabular_CPD(bnet, B, [0.01 0.99]);
%bnet.CPD{R} = tabular_CPD(bnet, R, [0.65 0.00001 0.35 0.99999]);
bnet.CPD{R} = tabular_CPD(bnet, R, [0.65 0.01 0.35 0.99]);
bnet.CPD{A} = tabular_CPD(bnet, A, [0.95 0.8 0.3 0.001 0.05 0.2 0.7 0.999]);
bnet.CPD{C} = tabular_CPD(bnet, C, [0.7 0.05 0.3 0.95]);


engine = jtree_inf_engine(bnet);
ev  = cell(1,N);
ev{C} = 1;
engine = enter_evidence(engine, ev);
mE = marginal_nodes(engine, E);
mB = marginal_nodes(engine, B);
fprintf('P(E|c)=%5.3f, P(B|c)=%5.3f\n', mE.T(1), mB.T(1))

ev{C} = 1;
ev{R} = 1;
engine = enter_evidence(engine, ev);
mE = marginal_nodes(engine, E);
mB = marginal_nodes(engine, B);
fprintf

P(E|c)=0.331, P(B|c)=0.077

P(E|c,r)=0.970, P(B|c,r)=0.029

In [9]:
do_file('brainy.m')

/home/jupyter/Notebooks/Classical/bnt/BNT/examples/static/brainy.m
% Example of explaining away from
% http://www.ai.mit.edu/~murphyk/Bayes/bnintro.html#explainaway
%
% Suppose you have to be brainy or smart to get into college.
% B S P(C=1) P(C=2)  1=false 2=true
% 1 1 1.0    0.0 
% 2 1 0.0    1.0
% 1 2 0.0    1.0
% 2 2 0.0    1.0
%
%
% If we observe that you are in college, you must be either brainy or sporty or both.
% If we observre you are in college and sporty, it is less likely you are brainy, 
% since brainy-ness and sporty-ness compete as causal explanations of the effect.

% B  S
%  \/
%   C

B = 1; S = 2; C = 3;
dag = zeros(3,3);
dag([B S], C)=1;
ns = 2*ones(1,3);
bnet = mk_bnet(dag, ns);
bnet.CPD{B} = tabular_CPD(bnet, B, 'CPT', [0.5 0.5]');
bnet.CPD{S} = tabular_CPD(bnet, S, 'CPT', [0.5 0.5]');
CPT = zeros(2,2,2);
CPT(1,1,:) = [1 0];
CPT(2,1,:) = [0 1];
CPT(1,2,:) = [0 1];
CPT(2,2,:) = [0 1];
bnet.CPD{C} = tabular_CPD(bnet, C, 'CPT', CPT);

engine = jtree_inf_engine(bnet);

P(B=true|C=true) = 0.667

P(B=true|C=true,S=true) = 0.500